In [84]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from keras.layers import Activation,Convolution2D,Dense,Dropout, merge, Reshape,Flatten,merge,Input,add,ZeroPadding2D,MaxPooling2D,AveragePooling2D
from keras.models import Sequential,Model
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.utils import np_utils
from sklearn.utils import shuffle
import operator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [2]:
df = pd.read_csv('./train.csv')
test_file = pd.read_csv('./test.csv')

In [3]:
df = df.values
test_file =test_file.values

In [4]:
split = int(0.80*df.shape[0])
x_train = df[:split,1:]
y_train = df[:split,0]
x_test = df[split:,1:]
y_test = df[split:,0]

print split
print x_train.shape,y_train.shape
print x_test.shape,y_test.shape

33600
(33600, 784) (33600,)
(8400, 784) (8400,)


In [5]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(x_test)
print y_train.shape,y_test.shape
print "---"
x_train = x_train.reshape((-1,28,28,1))
x_test = x_test.reshape((-1,28,28,1))
print x_test.shape,x_train.shape

(33600, 10) (8400, 784, 256)
---
(8400, 28, 28, 1) (33600, 28, 28, 1)


In [6]:
#normalisation
x_train = x_train/255
x_test = x_test/255
test_file = test_file/255

In [103]:
image_input = Input(shape=(32,32,3))

model = VGG16(weights='imagenet', include_top=False,input_tensor=image_input)

In [88]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 56, 56, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 56, 56, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 56, 56, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 28, 28, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 28, 28, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 28, 28, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 14, 14, 128)       0         
__________

In [94]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(512, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(10, activation='sigmoid'))
last_layer = top_model.layers[-1]


model2 = Model(output=top_model(model.output),inputs=model.input)
for layer in model2.layers[:-3]:
    layer.trainable = False
model2.summary()
#     return model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 56, 56, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 56, 56, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 56, 56, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 28, 28, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 28, 28, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 28, 28, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 14, 14, 128)       0         
__________

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  # This is added back by InteractiveShellApp.init_path()


In [95]:
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [96]:
model2.fit(x_train,y_train,epochs=2,shuffle=True,batch_size=128,validation_data=(x_test,y_test))

ValueError: Error when checking input: expected input_25 to have shape (56, 56, 3) but got array with shape (28, 28, 1)